# Challenge: Promotions

In this challenge, you'll develop codes to parse and analyze data returned from another API on Zalando such as [Promos homme (Men's Promotions)
](https://www.zalando.fr/promo-homme/) or [Promos femme (Women's Promotions)](https://www.zalando.fr/promo-femme/). The workflow is almost the same as in the guided lesson but you'll work with different data.

## Obtaining the link

Wrote your codes in the cell below to obtain the data from the API endpoint you choose. A recap of the workflow:

1. Examine the webpages and choose one that you want to work with.

1. Use Google Chrome's DevTools to inspect the XHR network requests. Find out the API endpoint that serves data to the webpage.

1. Test the API endpoint in the browser to verify its data.

1. Change the page number offset of the API URL to test if it's working.

In [1]:
url='https://www.zalando.fr/api/catalog/articles?categories=promo-homme&limit=84&offset=84&sort=sale'

## Reading the data

In the next cell, use Python to obtain data from the API endpoint you chose in the previous step. Workflow:

1. Import libraries.

1. Define the initial API endpoint URL.

1. Make request to obtain data of the 1st page. Flatten the data and store it in an empty object variable.

1. Find out the total page count in the 1st page data.

1. Use a FOR loop to make requests for the additional pages from 2 to page count. Append the data of each additional page to the flatterned data object.

1. Print and review the data you obtained.

In [2]:
# Imports
import json
import requests
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize

In [3]:
# headers definition
headers = {'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/75.0.3770.90 Chrome/75.0.3770.90 Safari/537.36'}

In [4]:
response = requests.get(url, headers = headers)
results = response.json()
results

{'total_count': 42963,
 'pagination': {'page_count': 512, 'current_page': 2, 'per_page': 84},
 'sort': 'sale',
 'articles': [{'sku': 'ZA812C001-H11',
   'name': 'Mocassins - orange',
   'price': {'original': '30,99\xa0€',
    'promotional': '24,80\xa0€',
    'has_different_prices': False,
    'has_different_original_prices': False,
    'has_different_promotional_prices': False,
    'has_discount_on_selected_sizes_only': False},
   'sizes': ['41', '42', '43', '44', '45'],
   'url_key': 'zalando-essentials-mocassins-orange-za812c001-h11',
   'media': [{'path': 'ZA/81/2C/00/1H/11/ZA812C001-H11@8.jpg',
     'role': 'DEFAULT',
     'packet_shot': True}],
   'brand_name': 'Zalando Essentials',
   'is_premium': False,
   'family_articles': [{'sku': 'ZA812C001-H11',
     'url_key': 'zalando-essentials-mocassins-orange-za812c001-h11',
     'media': [{'path': 'ZA/81/2C/00/1H/11/ZA812C001-H11@8.jpg',
       'role': 'FAMILY',
       'packet_shot': True}],
     'name': 'Mocassins - orange',
     's

In [5]:
flattened_data = json_normalize(results)
flattened_data.head()

,articles,articlesToShow,breadcrumbs,carouselTeaser,categoryTree,collection,contentPositions.entry-point-teasers,contentPositions.in-cat-carousel,contentPositions.in-cat-carousel-fullwidth,contentPositions.in-cat-carousel-mobile,...,total_article_count,total_count,upperInCatTeaser,variants.fullWidthCatalog,variants.hideCategories,variants.mobileLightFilters,variants.myBrandsFilter,variants.outwardTeaserCard,variants.premiumCatalog,wishlist
0,"[{'sku': 'ZA812C001-H11', 'name': 'Mocassins -...",84,"[{'items': [{'label': 'Homme', 'url_key': 'hom...",None,"[{'label': 'Promotions', 'id': '9191', 'url_ke...",None,"[7, 14, 20, 26]",9,8,6,...,42988,42963,None,False,False,False,True,False,False,None


In [6]:
articles = json_normalize(flattened_data.articles[0])
articles.head()

,brand_name,family_articles,flags,is_premium,media,name,outfits,price.has_different_original_prices,price.has_different_prices,price.has_different_promotional_prices,price.has_discount_on_selected_sizes_only,price.original,price.promotional,product_group,sizes,sku,url_key
0,Zalando Essentials,"[{'sku': 'ZA812C001-H11', 'url_key': 'zalando-...","[{'key': 'discountRate', 'value': '-20%', 'tra...",False,[{'path': 'ZA/81/2C/00/1H/11/ZA812C001-H11@8.j...,Mocassins - orange,NaN,False,False,False,False,"30,99 €","24,80 €",shoe,"[41, 42, 43, 44, 45]",ZA812C001-H11,zalando-essentials-mocassins-orange-za812c001-h11
1,Jack & Jones,"[{'sku': 'JA222O2DF-A11', 'url_key': 'jack-and...","[{'key': 'discountRate', 'value': '-60%', 'tra...",False,[{'path': 'JA/22/2O/2D/FA/11/JA222O2DF-A11@4.j...,JCORIO TEE CREW NECK - T-shirt imprimé - white,NaN,False,False,False,False,"17,95 €","7,18 €",clothing,"[S, M, L, XL, XXL]",JA222O2DF-A11,jack-and-jones-jcorio-tee-crew-neck-t-shirt-im...
2,Versus Versace,"[{'sku': 'VE052L00M-F11', 'url_key': 'versus-v...","[{'key': 'discountRate', 'value': '-20%', 'tra...",True,[{'path': 'VE/05/2L/00/MF/11/VE052L00M-F11@6.j...,Bague - gold-coloured,NaN,False,False,False,False,"129,95 €","104,00 €",accessoires,"[M, L]",VE052L00M-F11,versus-versace-bague-gold-coloured-ve052l00m-f11
3,Calvin Klein,"[{'sku': '6CA12C00F-K11', 'url_key': 'calvin-k...","[{'key': 'discountRate', 'value': '-30%', 'tra...",True,[{'path': '6C/A1/2C/00/FK/11/6CA12C00F-K11@12....,IVAN - Mocassins - dark navy,NaN,False,False,False,False,"119,95 €","83,95 €",shoe,"[41, 43, 44, 45, 46]",6CA12C00F-K11,calvin-klein-ivan-mocassins-6ca12c00f-k11
4,Reebok Classic,"[{'sku': 'RE012B02E-Q12', 'url_key': 'reebok-c...","[{'key': 'discountRate', 'value': 'Jusqu’à -40...",False,[{'path': 'RE/01/2B/02/EQ/12/RE012B02E-Q12@12....,EXOFIT - Baskets basses - black/silver,NaN,False,True,True,False,"79,95 €","47,95 €",shoe,"[35, 36, 36.5, 37.5, 38.5, 39, 40, 40.5, 41, 4...",RE012B02E-Q12,reebok-classic-exofit-baskets-basses-re012b02e...


In [7]:
articles.shape

(84, 17)

In [8]:
total_pages = results['pagination']['page_count']

df = pd.DataFrame()
for i in range(total_pages):
    art_per_page = i*84
    url_loop = 'https://www.zalando.fr/api/catalog/articles?categories=promo-homme&limit=84&offset='+str(art_per_page)+'&sort=sale'
    results_loop = requests.get(url_loop, headers = headers).json()
    flattened_data_loop = json_normalize(results_loop)
    articles_loop = json_normalize(flattened_data_loop.articles[0])
    df = df.append(articles_loop)

/home/eiroamd/.local/lib/python3.6/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [9]:
df.head()

,amount,brand_name,family_articles,flags,is_premium,media,name,outfits,price.base_price,price.has_different_original_prices,...,price.has_discount_on_selected_sizes_only,price.original,price.promotional,product_group,sizes,sku,tracking_information.impression_beacon,tracking_information.metrigo_impression_urls,tracking_information.source,url_key
0,NaN,The North Face,"[{'sku': 'TH342E038-Q11', 'url_key': 'the-nort...","[{'key': 'sponsored', 'value': 'Sponsorisé', '...",False,[{'path': 'TH/34/2E/03/8Q/11/TH342E038-Q11@4.1...,Pantalon de survêtement - black,NaN,NaN,False,...,False,"85,00 €","76,50 €",clothing,"[XS, S, M, L, XL, XXL]",TH342E038-Q11,https://ccp-et.metrigo.zalan.do/event/sbv?z=a4...,[https://ccp-et.metrigo.zalan.do/event/sbv?z=a...,ccp,the-north-face-pantalon-de-survetement-black-t...
1,160 g,Eastpak,"[{'sku': 'EA254H0LT-K11', 'url_key': 'eastpak-...","[{'key': 'sponsored', 'value': 'Sponsorisé', '...",False,[{'path': 'EA/25/4H/0L/TK/11/EA254H0LT-K11@17....,Sac banane - blue,NaN,NaN,False,...,False,"60,00 €","42,00 €",accessoires,[One Size],EA254H0LT-K11,https://ccp-et.metrigo.zalan.do/event/sbv?z=a4...,[https://ccp-et.metrigo.zalan.do/event/sbv?z=a...,ccp,eastpak-sac-banane-blue-ea254h0lt-k11
2,NaN,Desigual,"[{'sku': 'DE122O031-A11', 'url_key': 'desigual...","[{'key': 'sponsored', 'value': 'Sponsorisé', '...",False,[{'path': 'DE/12/2O/03/1A/11/DE122O031-A11@6.1...,T-shirt imprimé - white,NaN,NaN,False,...,False,"49,95 €","34,97 €",clothing,"[S, M, L, XL, XXL]",DE122O031-A11,https://ccp-et.metrigo.zalan.do/event/sbv?z=a4...,[https://ccp-et.metrigo.zalan.do/event/sbv?z=a...,ccp,desigual-t-shirt-imprime-white-de122o031-a11
3,NaN,Nike Sportswear,"[{'sku': 'NI112O07Y-Q11', 'url_key': 'nike-spo...","[{'key': 'campaign', 'value': 'HOT DROP', 'tra...",False,[{'path': 'NI/11/2O/07/YQ/11/NI112O07Y-Q11@11....,REACT ELEMENT 55 - Baskets basses - black/flas...,NaN,NaN,False,...,False,"129,95 €","71,00 €",shoe,"[38.5, 39, 40, 41, 42, 42.5, 43, 44, 44.5, 45,...",NI112O07Y-Q11,NaN,NaN,NaN,nike-sportswear-react-element-55-baskets-basse...
4,NaN,Nike Sportswear,"[{'sku': 'NI112O07Y-A11', 'url_key': 'nike-spo...","[{'key': 'campaign', 'value': 'HOT DROP', 'tra...",False,[{'path': 'NI/11/2O/07/YA/11/NI112O07Y-A11@8.j...,REACT ELEMENT 55 - Baskets basses - white/volt...,NaN,NaN,False,...,False,"129,95 €","77,95 €",shoe,"[39, 40, 40.5, 41, 42, 42.5, 43, 44, 44.5, 45,...",NI112O07Y-A11,NaN,NaN,NaN,nike-sportswear-react-element-55-baskets-basse...


## Bonus

Extract the following information from the data:

* The trending brand.

* The product(s) with the highest discount.

* The sum of discounts of all goods (sum_discounted_prices divided by sum_original_prices).

#### Trending brand:

In [10]:
df['brand_name'].value_counts().head()

YOURTURN              1187
Pier One              1119
Jack & Jones           886
Superdry               840
adidas Performance     774
Name: brand_name, dtype: int64

#### Product(s) with the highest discount:

In [11]:
# Defining a function to 'clean' the data and convert to float:
def stringcleaner(df, series):
    df[series] = [i.strip('€').strip() for i in df[series]]
    df[series] = df[series].str.replace(',','.')
    df[series] = pd.to_numeric(df[series],errors='coerce',downcast='float')

In [12]:
stringcleaner(df, 'price.original')

In [14]:
stringcleaner(df, 'price.promotional')

In [16]:
df['discount'] = df['price.original'] - df['price.promotional']

In [17]:
pivot_discount = pd.pivot_table(df, values=['discount'], 
                                index=['brand_name','name']).sort_values(by='discount',
                                                            ascending=False)
pivot_discount.head()

,,discount
brand_name,name,
Mackintosh,Trench - black,697.0
Missoni,PANTALONE - Pantalon classique - navy,672.0
John Richmond,JACKET STANHOPE - Veste de costume - black,665.0
Versace Collection,GIACCA FORMALE - Veste de costume - nero,651.0
Mackintosh,Manteau classique - grey,648.0


#### Sum of discounts of all goods (sum_discounted_prices divided by sum_original_prices):

In [18]:
df['discount'].sum()

1383299.6

In [19]:
a = df['price.promotional'].sum()
b = df['price.original'].sum()
c = a/b

In [20]:
print('sum_discounted_prices:', a)
print('sum_original_prices:', b)
print('division:', c.round(2))

sum_discounted_prices: 2295295.0
sum_original_prices: 3669740.8
division: 0.63
